In [1]:
#!/usr/bin/env env_name

In [3]:
!pip3 install matplotlib

     |████████████████████████████████| 8.5 MB 6.9 MB/s            
     |████████████████████████████████| 3.0 MB 28.8 MB/s            
     |████████████████████████████████| 61 kB 22.9 MB/s            
     |████████████████████████████████| 97 kB 12.5 MB/s            


In [6]:
from pyspark import SparkConf, SparkContext
conf = SparkConf().setAppName("App")
conf = (conf.setMaster('local[*]')
        .set('spark.executor.memory', '8G')
        .set('spark.driver.memory', '8G')
        .set('spark.driver.maxResultSize', '10G'))
sc = SparkContext(conf=conf)
# sc = pyspark.SparkContext().getOrCreate()

/Users/jackxu/opt/anaconda3/envs/env_name/lib/python3.6/site-packages/pyspark/context.py:238: FutureWarning: Python 3.6 support is deprecated in Spark 3.2.
  FutureWarning


In [1]:
from file_parse import FileParser
from transaction import *
from utils import *
from header import *
from get_block_uilts import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

## Parse Price Data
Parsing hourly btc usd data

In [33]:
btc_usd = pd.read_csv("../data/Bittrex_BTCUSD_1h.csv", header=0)
btc_usd = btc_usd.set_index('Unix Timestamp')
btc_usd

,Date,Symbol,Open,High,Low,Close,Volume BTC,Volume USD
Unix Timestamp,,,,,,,,
1.605856e+09,2020-11-20 07-AM,BTCUSD,18168.59,18390.00,18100.00,18194.27,38.38,701023.30
1.605852e+09,2020-11-20 06-AM,BTCUSD,18131.05,18187.32,18081.10,18168.59,12.34,223894.64
1.605848e+09,2020-11-20 05-AM,BTCUSD,18201.11,18227.37,18059.67,18131.05,22.04,399275.36
1.605845e+09,2020-11-20 04-AM,BTCUSD,17956.22,18234.99,17945.16,18201.11,41.49,751712.38
1.605841e+09,2020-11-20 03-AM,BTCUSD,17972.49,18028.19,17954.06,17956.22,14.29,257112.53
...,...,...,...,...,...,...,...,...
1.498921e+09,2017-07-01 03-PM,BTCUSD,2373.00,2399.10,2352.00,2397.00,99.25,235892.48
1.498918e+09,2017-07-01 02-PM,BTCUSD,2375.00,2393.43,2360.00,2373.00,130.11,308465.62
1.498914e+09,2017-07-01 01-PM,BTCUSD,2400.11,2460.00,2375.00,2375.00,112.07,271828.69


## Stream File Parser Workflow

To download and parse into list of samples at the same time

In [54]:
start_date = time.strptime('2020-01-01', '%Y-%m-%d')
end_date = time.strptime('2020-01-15', '%Y-%m-%d')

block = blocks_hashlist_by_time(start_date, end_date)

Start: 2020-01-01 End: 2020-01-15


In [55]:
block[:5]

[(612866, '0000000000000000000bb898960333faaecc81c30217d5b14b6f5d5e94ed2361'),
 (612865, '0000000000000000000a013f8b18168f63df8a65a19a239a029a15c106872667'),
 (612864, '0000000000000000000650bc59cd17d268dfaa1c6768fd890d32c1c7b38503d2'),
 (612863, '0000000000000000000aacba541ebb7b56b0831e4ae33faf20ff1e528bb9a657'),
 (612862, '00000000000000000003b7749bfd517d2672c6eace8c880794d47dc9040238ba')]

In [56]:
def avg(x):
    return [x[0]*3600, x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]

def matchPrice(x):
    return [x[0], x[1], x[2], x[3], btc_usd.loc[x[0]]['Close']]

In [67]:
blk_hash = sc.parallelize(block[:500])
fp = FileParser()

test = blk_hash.flatMap(lambda blk: 
                    fp.parse(FileLoader(blocks_blk_by_hash(blk[1], blk[0]))))
test = test.map(lambda x: (x[0]//3600, (x[1], x[2], x[3], x[4], 1)))
test = test.reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3], a[4]+b[4]))
test = test.map(lambda x: 
                matchPrice(avg(x)))
res = test.collect()

In [68]:
res

[[1579003200, 387212786.0, 585298722.1315109, 1.962319911340968, 8499.62],
 [1578960000, 387212786.0, 193310224.1986699, 2.5560540483479364, 8260.82],
 [1578916800, 387212786.0, 211303798.45708197, 2.5082906039821538, 8084.74],
 [1578873600, 387212786.0, 133464931.84772806, 1.8346340617412418, 8149.52],
 [1578830400, 387212786.0, 149680873.58245516, 1.9371418007025327, 8133.11],
 [1578787200, 387212786.0, 189318757.4486039, 1.8640007046595612, 8065.02],
 [1579006800, 387212786.0, 341445116.77835023, 2.4867558940471524, 8582.27],
 [1578963600, 387212786.0, 356840467.7865497, 2.230409356725146, 8425.82],
 [1578920400, 387212786.0, 220546414.37579176, 2.4093408505603118, 8078.23],
 [1578877200, 387212786.0, 256110211.20331362, 1.8733727810650889, 8145.72],
 [1578834000, 387212786.0, 226140537.14479855, 2.0583787922289214, 8155.09],
 [1578790800, 387212786.0, 313316483.8042376, 1.7264801297648014, 8063.39],
 [1579010400, 387212786.0, 453552071.37782246, 2.3834505862646567, 8701.55],
 [1578

In [69]:
res = np.array(res)
np.savetxt("../data/sample.txt", res)

## Disk File Parser Workflow

To parse local blk file into list of samples and output to local 

In [ ]:
fp = FileParser()
fpath = "../data/Blocks_612866_610546/*.blk"

# [time, nBits, value, in_count, out_count]
raw_data = sc.textFile(fpath)

In [ ]:
raw_data.flatMap(lambda x: fp.parse(FileLoader(x))).map(lambda x: (x[0]//3600, (x[1], x[2], x[3], x[4], 1))).take(5)

In [ ]:
test = raw_data.flatMap(lambda x: fp.parse(FileLoader(x))).map(lambda x: (x[0]//3600, (x[1], x[2], x[3], x[4], 1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3], a[4]+b[4])).take(2)

In [ ]:
test

In [ ]:
def avg(x):
    print(x)
    return [x[0]*3600, x[1][0]/x[1][4], x[1][1]/x[1][4], x[1][2]/x[1][4], x[1][3]/x[1][4]]

In [ ]:
res = raw_data.flatMap(lambda x: fp.parse(FileLoader(x))).map(lambda x: (x[0]//3600, (x[1], x[2], x[3], x[4], 1))).reduceByKey(lambda a,b: (a[0]+b[0], a[1]+b[1], a[2]+b[2], a[3]+b[3], a[4]+b[4])).map(lambda x: avg(x)).collect()

In [ ]:
res = np.array(res)
print(res)
# [time, nBits, value, in_count, out_count]
np.savetxt("../data/sample.txt", res)

In [ ]:
with open('../data/block706515.blk','r') as in_file:
    fd = FileLoader(in_file.readline())
    output = fp.parse(fd)
output

In [ ]:
with open('../data/block706515.blk','r') as in_file:
    fd = FileLoader(in_file.readline())
    output = fp.parse(fd)
output